In [4]:
# add popluation to Toronto neighborhood geojson file

import json

def modify_json():
    
    with codecs.open('data//toronto_chinese.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
    ch_popu = []
    for x in lines:
        item = {}
        item['area_id'] = x.split('\t')[0]
        item['popu'] = x.split('\t')[1]
        item['area_name'] = x.split('\t')[2]
        ch_popu.append(item)
        
#     print ch_popu
        
    with codecs.open('data//toronto_neighbors_geo.json', 'r', 'utf-8') as f:
        data = json.load(f)
        
    print type(data)
    cnt = 0
    for area in data['features']:        
        # chang key name from 'AREA_NAME' to 'name'
        area['properties']['name'] = area['properties'].pop('AREA_NAME')
        area_id = area['properties']['AREA_S_CD']
        for x in ch_popu:
            if area_id.lstrip('0') == x['area_id']:                
                cnt += 1
                area['properties']['density'] = x['popu']
                #area['properties']['name'] = x['area_name']
        print area['properties']
    print cnt
    
    fd = codecs.open('data\\to_neighbors.js', 'w', 'utf-8')
    fd.write('var areaData = ')
    json.dump(data, fd)
    fd.close()
    
modify_json()


<type 'dict'>
{u'AREA_S_CD': u'097', 'name': u'Yonge-St.Clair (97)', 'density': u'250'}
{u'AREA_S_CD': u'027', 'name': u'York University Heights (27)', 'density': u'1750'}
{u'AREA_S_CD': u'038', 'name': u'Lansing-Westgate (38)', 'density': u'995'}
{u'AREA_S_CD': u'031', 'name': u'Yorkdale-Glen Park (31)', 'density': u'640'}
{u'AREA_S_CD': u'016', 'name': u'Stonegate-Queensway (16)', 'density': u'330'}
{u'AREA_S_CD': u'118', 'name': u"Tam O'Shanter-Sullivan (118)", 'density': u'8220'}
{u'AREA_S_CD': u'063', 'name': u'The Beaches (63)', 'density': u'245'}
{u'AREA_S_CD': u'003', 'name': u'Thistletown-Beaumond Heights (3)', 'density': u'95'}
{u'AREA_S_CD': u'055', 'name': u'Thorncliffe Park (55)', 'density': u'215'}
{u'AREA_S_CD': u'059', 'name': u'Danforth East York (59)', 'density': u'1095'}
{u'AREA_S_CD': u'106', 'name': u'Humewood-Cedarvale (106)', 'density': u'220'}
{u'AREA_S_CD': u'014', 'name': u'Islington-City Centre West (14)', 'density': u'1060'}
{u'AREA_S_CD': u'066', 'name': u'

In [6]:
import codecs
from collections import Counter

def get_district(a):
    return a.split(';')[9]

def clean_data():
    '''two ways to sort a list by some feature: normal function and lambda'''
    with codecs.open('data\\church_20151220_v1.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
    
    fd = codecs.open('data\\church_20151220.txt', 'w', 'utf-8')
    
    title = lines[0].split(';')
    # swap 0th and 9th coloumn
    title.insert(0, title.pop(9))
    fd.write(';'.join(x for x in title) + '\n')
    
    
    #new_lines = sorted(lines, key=get_district)
        
    # sort by frequency of districts
    # a catch: for districts with same frequency, the sorted result may not be what you want
    district_cnt = Counter([x.split(';')[9] for x in lines[1:]])
    print district_cnt.most_common()    
    new_lines = sorted(lines[1:], \
                                key=lambda x: str([i[1] for i in district_cnt.most_common() \
                                   if x.split(';')[9] == i[0]][0] + 127) + x.split(';')[9], reverse=True)
    
    #new_lines = sorted(lines[1:], key=lambda x: x.split(';')[9], reverse=True)  # lambda, sorted by district name
    
    # move District to the first column

    cnt = 0
    for x in new_lines:
        cnt += 1
        y = [x_i.strip() for x_i in x.split(';')]
        y.insert(0, y.pop(9))
        fd.write(';'.join(y_i for y_i in y) + '\n')
        
    fd.close()

clean_data()


[(u'Scarborough', 36), (u'Markham', 34), (u'North York', 23), (u'Mississauga', 17), (u'Old Toronto', 12), (u'Richmond Hill', 9), (u'East York', 4), (u'Brampton', 2), (u'Oakville', 2), (u'Newmarket', 2), (u'Etobicoke', 2), (u'Pickering', 1), (u'Guelph', 1), (u'Whitchurch-Stouffville', 1), (u'Barrie', 1), (u'Burlington', 1), (u'Whitby', 1), (u'Niagara Falls', 1), (u'Aurora', 1)]


In [27]:

def clean_church_data():
    '''two ways to sort a list by some feature: normal function and lambda'''
    with codecs.open('data\\church_cleaner.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
    
    fd = codecs.open('data\\church_201512.txt', 'w', 'utf-8')
    fd.write(lines[0] + '\n')
    
    #new_lines = sorted(lines, key=get_district)
    #new_lines = sorted(lines[1:], key=lambda x: x.split(';')[9], reverse=True)  # lambda
    for x in lines[1:]:        
        fd.write(';'.join([x_i.strip() for x_i in x.split(';')[:6]]) + ';;;' + \
                 ';'.join([x_i.strip() for x_i in x.split(';')[6:]]) + '\n')
        
    fd.close()

In [15]:
# clean data

import codecs, re
dataname = 'organization'

with codecs.open('data\\' + dataname + '_raw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
# for x in lines[:10]: print x
fd = codecs.open('data\\' + dataname + '_clean.txt', 'w', 'utf-8')
    
cnt = 0
valid_input_cnt = 0

for x in lines:    
    cnt += 1
    item = {}
    cc = x.split('\t')
    if len(cc) < 2: continue
    #print x
    
    valid_input_cnt += 1
    
    c1, c2 = cc[0], cc[1]
    
    names = c1.split(',')
    chinese_name = ''
    english_name = ''
    for x in names:    
        if not re.search('[a-zA-Z]', x[-3:-1]):
            chinese_name += x
        else:
            english_name += x
    fd.write(chinese_name + ';' + english_name + ';')
    
    # check if the address is a PO Box mailing address
    mail_flag = 0
    if c2.find('Mailing') != -1 or c2.find('Box') != -1: mail_flag = 1    
    # all other info except for names
    alli = c2.split(',')
        
    
    phone = ''
    weburl = ''    
    email = ''
    addr = ''
    other = ''
    for y in alli:
#         print 'y: ', y

        # phone
        if y.find('(') != -1:        
            phone += y.replace(' ', '')
        # website
        elif y.find('www.') != -1:
            weburl += y.replace(' ', '')
        # email
        elif y.find('@') != -1:
            email += y.replace(' ', '')
        else:
            if mail_flag == 1:     
            # other
                other += y   
            else:
                # address
                #if y.find('(') == -1 and y.find('www.') == -1 and y.find('@') == -1 and y.find('Mailing'):
        #             print '-----this is part of address:'            
                    addr +=  y
    
    if phone: fd.write(phone + ';')
    else: fd.write(';')
        
    if weburl: fd.write(weburl + ';')
    else: fd.write(';')
        
    if email: fd.write(email + ';')
    else: fd.write(';')
    
    if addr: fd.write(addr.strip() + ';')
    else: fd.write(';')
        
    if other: fd.write(other.strip() + ';')
    else: fd.write(';')  
    
    # note field
    fd.write(';')  
            
    # updated address field
    fd.write(';\n')
        
fd.close()   

print 'Total lines: ', cnt, '; valide lines: ', valid_input_cnt


Total lines:  90 ; valide lines:  83


In [6]:
# Use Google Geocode API to get latitudes, longitudes, sublocality name, and formatted addresses

# import simplejson
import json, urllib, time

GEOCODE_BASE_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

def geocode(address, **geo_args):
    geo_args.update({
        'address': address
    })

    url = GEOCODE_BASE_URL + '?' + urllib.urlencode(geo_args)
    result = json.load(urllib.urlopen(url))
    print geo_args, ': ', result['status']
    
    return result  # type: dictionary

#         latlng = [lat, lng]
#     return latlng, addr, subloc   



def church_org():
#     test_addr = ["201 Cliffwood Road  North York  ON  L4B 3L4", "68 Revlis Cres  Toronto  ON  M1V 1E9", 
#              "206 McCaul Street  Toronto  ON  M5T 1W5", "670 Eglinton Ave E, Toronto, ON",
#              "UNIT 223-228 80-E  Nashdene Rd    Scarborough  ON  M1V 5E4"]

    dataname = 'organization'

    with codecs.open('data\\' + dataname + '_clean.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
        
    fgeo = codecs.open('data\\' + dataname + '_geocode.txt', 'w', 'utf-8')
    fd = codecs.open('data\\' + dataname + '_cleaner.txt', 'w', 'utf-8')
    
    # title line
    fd.write("Chinese Name;English Name;Phone;Website;E-mail;Original Address;Other Info;Note;Updated Address;")
    fd.write("District;Google Formatted Address;Latitude;Longitude\n")
    
    cnt = 0
    for line in lines:
        cnt += 1
        print cnt
        time.sleep(1)
        x = line.split(';')
        fd.write(line)
        
        addr = ''
        if x[8]:
            print '--- changed address', x[8]
            addr = x[8]
        else: 
            print x[5]
            addr = x[5]
        
        if addr:
            result = geocode(address = addr)
#             print type(result) # dict
#             print result
        else:
            print 'no input address\n'
            fd.write(';;;\n')
            continue
            
        json.dump(result, fgeo, indent=4)
        
        if result['status'] != 'OK':
            fd.write("Not a vaild address for Google Map;;;\n")
            continue
        
#     print json.dumps(result, indent=4, separators=(',', ': '))
    
        for x in result['results']:
            lat = x['geometry']['location']['lat']
            lng = x['geometry']['location']['lng']
            formatted_addr = x['formatted_address']
            subloc = ''
            for a in x["address_components"]:
                if "sublocality" in a['types']:
                    subloc = a["long_name"]
                    break
                elif 'locality' in a['types']: subloc = a["long_name"]
            print formatted_addr, ': ', lat, lng, subloc
            print '\n'
            fd.write(subloc + ';' + formatted_addr + ';' + str(lat) + ';' + str(lng) + '\n' )

    fd.close()
    fgeo.close()    
    

def school():

    dataname = 'Toronto_school'

    with codecs.open('data\\' + dataname + '.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
        
    fgeo = codecs.open('data\\' + dataname + '_geocode.txt', 'w', 'utf-8')
    fd = codecs.open('data\\' + dataname + '_cleaner.txt', 'w', 'utf-8')
    
        
    cnt = 0
    for line in lines:
        cnt += 1
        print cnt
        time.sleep(1)
        x = line.split(';')
        fd.write(line)
        
        addr = ''
        if x[1]:
            print x[1]
            addr = x[1]
        else: 
            print 'no address'
            continue
        
        if addr:
            result = geocode(address = addr)
#             print type(result) # dict
#             print result
        else:
            print 'no input address\n'
            fd.write('\n')
            continue
            
        json.dump(result, fgeo, indent=4)
        
        if result['status'] != 'OK':
            fd.write("Not a vaild address for Google Map\n")
            continue
        
    
        for x in result['results']:
            lat = x['geometry']['location']['lat']
            lng = x['geometry']['location']['lng']
            formatted_addr = x['formatted_address']
            subloc = ''
            for a in x["address_components"]:
                if "sublocality" in a['types']:
                    subloc = a["long_name"]
                    break
                elif 'locality' in a['types']: subloc = a["long_name"]
            print formatted_addr, ': ', lat, lng, subloc
            print '\n'
            fd.write(';' + subloc + ';' + formatted_addr + ';' + str(lat) + ';' + str(lng) + '\n' )

    fd.close()
    fgeo.close()      
    
if __name__ == '__main__':
    #school()
    result = geocode(address='4350 Steeles Ave East Markham ON L3R 9V4')
    print json.dumps(result, indent=4)
    
        

{'address': '4350 Steeles Ave East Markham ON L3R 9V4'} :  OK
{
    "status": "OK", 
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 43.8254466, 
                    "lng": -79.3050432
                }, 
                "viewport": {
                    "northeast": {
                        "lat": 43.8267955802915, 
                        "lng": -79.3036942197085
                    }, 
                    "southwest": {
                        "lat": 43.8240976197085, 
                        "lng": -79.30639218029151
                    }
                }, 
                "location_type": "ROOFTOP"
            }, 
            "formatted_address": "4350 Steeles Ave E, Markham, ON L3R 9V5, Canada", 
            "place_id": "ChIJPez8FwnU1IkRRWVgTnBNtyM", 
            "address_components": [
                {
                    "long_name": "4350", 
                    "types": [
                        "street_number"
 

In [3]:
# create ccefdata.js

import codecs, json

def create_marker_data_js(var_name, data_file):
    '''for church and organization data'''
    
    # for distributions of churches, organizations, and schools
    #fd = codecs.open('data\\ch_org_sch.js', 'a', 'utf-8')

    # for Toronto Chinese distributions
    fd = codecs.open('data\\church.js', 'a', 'utf-8')
    
    fd.write("var " + var_name + "=")
    
    with codecs.open('data\\' + data_file, 'r', 'utf-8') as f:
        lines = f.read().splitlines()
        
    all_stuff = []
    cnt = 0
    for line in lines[1:]:
        cnt += 1
        print cnt
        x = [a.strip() for a in line.split(';')]    
        item = {}
        
        if x[11] and x[12]:
            item['loc'] = [x[11], x[12]]
        else:
            print 'no lat lng data'
            continue
                
        descr = ''
        
        only_en_name = 0
        if x[1]:            
            descr += '<b>' + x[1] + '</b><br />'
            print x[1]
        elif x[2]:
            print 'only english name'
            print 'in elif', x[2]
            only_en_name = 1            
            descr += '<b>' + x[2] + '</b><br />'
        if only_en_name == 0 and x[2]:           
            print 'both name', x[2]
            descr += x[2] + '<br />'
            
        if x[3]:             
            descr += '<br />Phone: ' + x[3]
        if x[4]: 
            x4 = x[4].split(',')
            if len(x4) == 1:
                descr += "<br />Website: <a href='http://" + x[4] + "'  target='_blank'>" + x[4] + "</a>"
            elif len(x4) >= 2:
                for x4_i in x4:
                    descr += "<br />Website: <a href='http://" + x4_i + "'  target='_blank'>" + x4_i + "</a>"            
            #<a href='http://www.wocbc.ca'  target='_blank'>www.wocbc.ca</a>
        if x[5]: 
            descr += '<br />Email: ' + x[5]
        if x[0]: 
            descr += '<br />District: ' + x[0]
        if x[10]: 
            descr += '<br />Address: ' + x[10]
        
        print descr
        if descr: item['descr']= descr
            
        all_stuff.append(item)
        
    json.dump(all_stuff, fd)
    fd.write('\n')
    fd.close()

create_marker_data_js('c_loc', 'church_20151220_subset.txt')
# create_marker_data_js('c_loc', 'church_20151220.txt')
# create_marker_data_js('o_loc', 'organization_20151220.txt')


1
多倫多恆愛基督教會
both name Care & Support Fellowship Church
<b>多倫多恆愛基督教會</b><br />Care & Support Fellowship Church<br /><br />Phone: (416)412-7941<br />District: Scarborough<br />Address: 68 Revlis Crescent, Scarborough, ON M1V 1E9, Canada
2
only english name
in elif Faith Hope Love Ministry at Grace Church
<b>Faith Hope Love Ministry at Grace Church</b><br /><br />Phone: (416)999-3893<br />Email: faithhopelove54@gmail.com<br />District: Scarborough<br />Address: 25 Channel Nine Ct, Scarborough, ON M1S 4B5, Canada
3
多倫多台恩基督教會
both name Formosan Grace Christian Church In Toronto
<b>多倫多台恩基督教會</b><br />Formosan Grace Christian Church In Toronto<br /><br />Phone: (416)850-2270<br />Website: <a href='http://www.fgcct.org'  target='_blank'>www.fgcct.org</a><br />District: Scarborough<br />Address: 841 Progress Ave, Scarborough, ON M1H 2X4, Canada
4
仁愛福音堂
both name Agape Gospel Church
<b>仁愛福音堂</b><br />Agape Gospel Church<br /><br />Phone: (416)759-3773<br />District: Scarborough<br />Address: 250

In [ ]:
    
def create_school_js():
    dataname = 'Toronto_school'
    fd = codecs.open('data\\ccefdata.js', 'a', 'utf-8')
    fd.write("var s_loc = ")
    
    with codecs.open('data\\' + dataname + '_cleaner.txt', 'r', 'utf-8') as f:
        lines = f.read().splitlines()
        
    all_stuff = []
    cnt = 0
    for line in lines:
        cnt += 1
        print cnt
        x = [a.strip() for a in line.split(';')]    
        item = {}
        
        if x[5] and x[6]:
            item['loc'] = [x[5], x[6]]
        else:
            print 'no lat lng data'
            continue
                
        descr = ''
        
        if x[0]:            
            descr += '<b>' + x[0] + '</b><br />'
            print x[0]        
                    
        if x[2]: 
            descr += "<br />Website: <a href='http://" + x[2] + "'  target='_blank'>" + x[2] + "</a>"            
            #<a href='http://www.wocbc.ca'  target='_blank'>www.wocbc.ca</a>        
        if x[3]: 
            descr += '<br />District: ' + x[3]
        if x[4]: 
            descr += '<br />Address: ' + x[4]
        
        print descr
        if descr: item['descr']= descr
            
        all_stuff.append(item)
        
    json.dump(all_stuff, fd)
    fd.close()    
    
create_school_js()